In [4]:
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
# Aaronson 은 유죄인가요?
# 그가 테이블에 어떤 메시지를 썼나요?
# Julia 는 누구인가요?

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory #조건
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.text_splitter import CharacterTextSplitter 

from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini-2024-07-18"
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(  #tiktoken == 토큰화 
    separator="\n", #Line break
    chunk_size=600, #최대글자갯수
    chunk_overlap=100, #문장이나 문단을 분할할때, 앞 조각 일부분을 가져오게 만든다. 문제는 중복되는 부분이 생김. (문장구조는 안해침)
    # lenth_function = len, #t원래 oken != letter 
)

loader = UnstructuredFileLoader("./files/document.txt") 
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            주어진 글을 읽고, 질문에 대답하세요.
            -------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {
    "context": retriver, 
    "question": RunnablePassthrough(), 
    "history": load_memory
    } | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)


In [8]:
invoke_chain("Aaronson 은 유죄인가요?")

주어진 문서에서는 Aaronson에 대한 구체적인 정보나 그의 유죄 여부에 대한 언급이 없습니다. "1984" 소설의 맥락에서 Aaronson은 과거에 반당적 활동을 했던 인물로, 그가 유죄인지 여부는 당의 정치적 판단에 따라 결정됩니다. 소설에서는 그가 고문을 받고 유죄로 몰리게 되는 상황이 묘사되지만, 구체적인 세부사항은 문서에 포함되어 있지 않습니다. Aaronson의 유죄 여부에 대한 정확한 판단은 소설의 전체적인 맥락을 통해 이해해야 합니다.


In [6]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

Winston은 테이블에 다음과 같은 메시지를 썼습니다:

1. "FREEDOM IS SLAVERY" (자유는 노예다)
2. "TWO AND TWO MAKE FIVE" (2와 2는 5다)
3. "GOD IS POWER" (신은 권력이다)

이 메시지들은 그가 당의 이념과 사상을 받아들이고 있다는 것을 나타내는 상징적인 문구들입니다.


In [7]:
invoke_chain("Julia 는 누구인가요?")

Julia는 소설 "1984"의 주요 등장인물 중 하나로, Winston Smith의 연인입니다. 그녀는 당의 억압적인 체제에 반대하는 감정과 욕망을 가진 인물로, Winston과 함께 비밀스럽게 사랑을 나누며 자유와 개인적인 행복을 추구합니다. Julia는 당의 통제에 저항하려는 Winston에게 중요한 존재이며, 그들의 관계는 개인의 자유와 사랑의 의미를 탐구하는 중요한 요소로 작용합니다.
